In [2]:
from common_nba_api_functions import rate_limited_request, get_season_start_and_end
from common_db_functions import get_players_filtered_by_team_playoff_participation
from manipulation import get_date_n_days_ago

season = "2023-24"
season_type = "Playoffs"
season_start, season_end = get_season_start_and_end(season)
players = get_players_filtered_by_team_playoff_participation(season, season_type)
player_ids = [player.player_id for player in players]
end = get_date_n_days_ago(1)
start = get_date_n_days_ago(10)




[1628369, 201950, 1627759, 204001, 1628401, 1630202, 1629052, 1641809, 1631120, 1630214, 1641775, 1630573, 1628436, 201143, 1630531, 1629004, 1629674, 1629622, 1629660, 1627747, 1630596, 1630241, 1631250, 1641854, 1629636, 202684, 1630600, 1627777, 1641734, 202694, 1627745, 1628386, 1629731, 1630171, 1628378, 1631220, 1629627, 203468, 1630529, 1630230, 1630700, 1627742, 1630631, 202685, 1626204, 1641722, 1630530, 1630604, 1630792, 1630346, 203469, 1630526, 203957, 1630314, 1630702, 1641726, 203939, 1630182, 1630639, 203501, 202681, 1641788, 1641765, 1629655, 1629023, 1630535, 1628467, 1627884, 1629029, 202693, 1631128, 1629008, 1631124, 1641816, 203484, 201599, 202704, 1631212, 203200, 1630296, 203999, 1631221, 1630192, 1629618, 1627750, 1630643, 1628427, 203932, 1641945, 203081, 1630699, 1627736, 1641753, 1626171, 201572, 201568, 1631260, 201976, 203114, 1631102, 1626192, 203507, 203648, 1641748, 1631157, 203109, 1641740, 203937, 1630183, 1630162, 1629162, 1631111, 1631169, 1629638, 2

In [3]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.exc import SQLAlchemyError
from db_config import get_session
from models import GameStats, Player
from common_nba_api_functions import rate_limited_request
session = get_session()



def fetch_and_store_player_game_stats(player_id):
    try:
        player_name = session.query(Player).filter(Player.player_id == player_id).one_or_none().name
        print(f"Fetching data for player: {player_name}")
        gamelog = playergamelog.PlayerGameLog(
            player_id=player_id, 
            season=season, 
            season_type_all_star=season_type, 
            date_from_nullible=start, 
            date_to_nullible=end
            )
        games_df = gamelog.get_data_frames()[0]
        print(games_df.head())
        games = []
        for index, game in games_df.iterrows():
            print(f"Adding new game stat for game ID: {game['Game_ID']}")
            new_game_stat = GameStats(
                player_id=db_player.id,
                game_id=game['Game_ID'],
                points=game['PTS'],
                assists=game['AST'],
                rebounds=game['REB'],
                steals=game['STL'],
                blocks=game['BLK'],
                turnovers=game['TOV'],
                fouls=game['PF'],
                minutes=game['MIN'],
                fg_made=game['FGM'],
                fg_attempts=game['FGA'],
                fg_percentage=game['FG_PCT'],
                fg3_made=game['FG3M'],
                fg3_attempts=game['FG3A'],
                fg3_percentage=game['FG3_PCT'],
                ft_made=game['FTM'],
                ft_attempts=game['FTA'],
                ft_percentage=game['FT_PCT'],
                plus_minus=game['PLUS_MINUS']
            )
            games.append(new_game_stat)
        session.add_all(games)
        session.commit()

    except SQLAlchemyError as e:
        print(f"Database error: {e}")
        session.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # It's a good practice to ensure the session is closed properly
        session.close()

for player_id in player_ids:
    rate_limited_request(2)
    fetch_and_store_player_game_stats(player_id)

    # Close session
    session.close()

Fetching data for player: Jayson Tatum
  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP    WL  MIN  FGM  \
0     42023    1628369  0042300203  MAY 11, 2024    BOS @ CLE  None   19    9   
1     42023    1628369  0042300202  MAY 09, 2024  BOS vs. CLE     L   38    7   
2     42023    1628369  0042300201  MAY 07, 2024  BOS vs. CLE     W   35    7   
3     42023    1628369  0042300105  MAY 01, 2024  BOS vs. MIA     W   32    4   
4     42023    1628369  0042300104  APR 29, 2024    BOS @ MIA     W   37    5   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   18   0.500  ...    10   11    5    1    1    4   2   27           2   
1   17   0.412  ...     7    7    6    1    0    1   3   25         -22   
2   19   0.368  ...     9   11    5    0    3    1   3   18          18   
3    9   0.444  ...    12   12    3    0    1    2   2   16          35   
4   14   0.357  ...    11   11    5    0    0    1   3   20          12   

   VIDEO_AVAILABLE  
0 